In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("pyspark-joins") \
    .getOrCreate()

In [2]:
df_posts = spark.read.parquet("gs://bucket-name/spark-joins/post_questions/")
df_post_answers = spark.read.parquet("gs://bucket-name/spark-joins/post_answers/")

In [7]:
# Execute this block to check partitions
df_posts = df_posts.repartition(5)
df_posts.rdd.getNumPartitions()

In [9]:
# Execute this block to check partitions
df_posts = df_posts.coalesce(3)
df_posts.rdd.getNumPartitions()

In [11]:

# Execute this block to implement partitions
df_posts.repartition(5).write.parquet("gs://bucket-name/spark-joins/df-repartitioned/")

df_posts.coalesce(2).write.parquet("gs://bucket-name/spark-joins/df-coalesce/")

In [3]:
# Execute this block to check partitions
df_posts.rdd.getNumPartitionsumPartitions()

3

In [4]:
# Execute this block to check partitions
df_post_answers.rdd.getNumPartitions()

2

In [ ]:
df_posts.createOrReplaceTempView("posts")

df_post_answers.createOrReplaceTempView("post_answers")

In [ ]:
sql = """
    select 
        p.id ,
        p.score , 
        cast(p.creation_date as timestamp) as post_date,
        p.comment_count,
        pa.answer_id,
        cast(pa.creation_date as timestamp) as answer_date
    from 
        posts p
    join 
        post_answers pa 
    on 
        p.id = pa.post_id 
    limit 10
     """

spark.sql(sql).show(10,False)

In [ ]:
sql = """
    select 
        p.id ,
        p.score , 
        cast(p.creation_date as timestamp) as post_date,
        p.comment_count,
        pa.answer_id,
        cast(pa.creation_date as timestamp) as answer_date
    from 
        posts p
    full outer join 
        post_answers pa 
    on 
        p.id = pa.post_id 
    limit 10
     """

spark.sql(sql).show(10,False)

In [ ]:
df_posts.join(df_post_answers,df_posts.id == df_post_answers.post_id,"full_outer") \
.select(
    [
        df_posts.id.alias("post_id"),
        df_posts.score,
        df_posts.creation_date.alias("post_creation_date"),
        df_posts.comment_count.alias("post_comment_count")
    ]
).show(10,False)